In [1]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests  # You'll need this to make API calls for live data

# Function to fetch live crypto data
def fetch_crypto_data(coin_ids):
    data = []
    for coin in coin_ids:
        response = requests.get(f"https://api.coingecko.com/api/v3/coins/{coin}")
        coin_data = response.json()
        data.append({
            'name': coin_data['name'],
            'current_price': coin_data['market_data']['current_price']['usd'],
            'market_cap': coin_data['market_data']['market_cap']['usd'],
            'total_volume': coin_data['market_data']['total_volume']['usd'],
            'high_24h': coin_data['market_data']['high_24h']['usd']
        })
    return pd.DataFrame(data)

# Initialize the Dash app
app = Dash(__name__)

app.layout = html.Div([
    dcc.Interval(
        id='interval-update',
        interval=1*60*1000,  # Update every 5 minutes
        n_intervals=0
    ),
    dcc.Graph(id='live-graph')
])

@app.callback(
    Output('live-graph', 'figure'),
    Input('interval-update', 'n_intervals')
)
def update_graph(n):
    coin_ids = ["basic-attention-token", "cardano", "stellar", "vechain"]
    df = fetch_crypto_data(coin_ids)

    fig = make_subplots(rows=2, cols=2, specs=[[{'type': 'xy'}, {'type': 'xy'}], [{'type': 'pie'}, {'type': 'xy'}]],
                        subplot_titles=("Current Price in USD", "Market Cap", "Trading Volume", "24h High"))

    fig.add_trace(go.Bar(x=df['name'], y=df['current_price'], name='Current Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['name'], y=df['market_cap'], mode='lines+markers', name='Market Cap'), row=1, col=2)
    fig.add_trace(go.Pie(labels=df['name'], values=df['total_volume'], name='Trading Volume'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['name'], y=df['high_24h'], fill='tozeroy', name='24h High'), row=2, col=2)

    fig.update_layout(title='Crypto Dashboard')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
